### Setup & Config

In [2]:
import os, random, math, numpy as np
from collections import Counter
from typing import List, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Sampler

try:
    # torchvision v2 transforms (tensor-native)
    from torchvision.transforms import v2 as T
except:
    # fallback to classic, but v2 is recommended
    import torchvision.transforms as T

from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

# ----- Paths (edit these) -----
TRAIN_DIR = "D:/dataset/npz_80_tiny"                   # reduced, balanced-ish for training
VAL_DIR   = "D:/dataset/converted_classifier_npz_compact"  # full-length compacts for validation/inference
SAVE_PATH = "D:/acouslic-ai-cse4622/saved_weights/best_frame_classifier_allinone.pth"

# ----- Splits (edit indices as you like) -----
train_files = sorted([f for f in os.listdir(TRAIN_DIR) if f.endswith(".npz")])[:210]
val_files   = sorted([f for f in os.listdir(VAL_DIR)   if f.endswith(".npz")])[210:255]

# ----- Training hyperparams -----
IMAGE_SIZE = 224
BATCH_SIZE = 32
VAL_BATCH  = 64
EPOCHS     = 20
PATIENCE   = 5
LR_HEAD    = 1e-3     # warmup (head-only)
LR_ALL     = 1e-4     # full fine-tune
WEIGHT_DEC = 1e-4
SEED       = 42

device = "cuda" if torch.cuda.is_available() else "cpu"
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)



### Utilities: aspect-preserving letterbox

In [3]:
def _letterbox_to_square(x: torch.Tensor, size: int = 224) -> torch.Tensor:
    """
    x: (B,1,H,W) or (1,H,W) in [0,1] -> pad to square (keep aspect) -> resize to (..,1,size,size)
    """
    is_batched = (x.dim() == 4)
    if not is_batched:
        x = x.unsqueeze(0)  # (1,1,H,W)
    _, _, H, W = x.shape
    s = max(H, W)
    pad_h = (s - H) // 2
    pad_w = (s - W) // 2
    x = F.pad(x, (pad_w, s - W - pad_w, pad_h, s - H - pad_h))  # L,R,T,B
    x = F.interpolate(x, size=(size, size), mode="bilinear", align_corners=False)
    return x if is_batched else x.squeeze(0)

### 2) Dataset with tensor transforms (binary)

In [4]:
class PreloadedNPZFrameDataset(Dataset):
    """
    Preloads all frames from the given NPZ files into RAM as a single tensor.
    binary=True merges class 2->1 (optimal+suboptimal).
    resize_mode: 'letterbox' (preserves aspect) or 'squash' (direct resize).
    dtype=torch.float16 to save RAM (use AMP on GPU).
    """
    def __init__(self,
                 npz_dir: str,
                 files: list[str],
                 binary: bool = True,
                 out_size: int = 224,
                 resize_mode: str = "letterbox",
                 dtype: torch.dtype = torch.float16):
        self.binary = binary
        self.out_size = out_size
        self.resize_mode = resize_mode
        self.dtype = dtype

        imgs_all = []
        labels_all = []

        for f in files:
            path = os.path.join(npz_dir, f)
            case = np.load(path, allow_pickle=True)
            imgs = case["image"]                    # (T,H,W) uint8
            y    = case["label"].astype(np.int64)   # (T,)
            if binary:
                y[y == 2] = 1

            t = torch.from_numpy(imgs).unsqueeze(1).float() / 255.0  # (T,1,H,W) [0,1]

            if resize_mode == "letterbox":
                # letterbox each frame (simple loop is reliable & clear)
                out_frames = []
                for fr in t:
                    out_frames.append(_letterbox_to_square(fr, size=out_size))  # (1,S,S)
                t = torch.stack(out_frames, dim=0)  # (T,1,S,S)
            else:  # 'squash'
                t = F.interpolate(t, size=(out_size, out_size), mode="bilinear", align_corners=False)

            # normalize to [-1,1]
            t = (t - 0.5) / 0.5
            imgs_all.append(t.to(dtype=dtype).cpu())
            labels_all.append(torch.from_numpy(y).long())

        self.images = torch.cat(imgs_all, dim=0)   # (N,1,S,S) FP16 by default
        self.labels = torch.cat(labels_all, dim=0) # (N,)

    def __len__(self):
        return self.labels.numel()

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

### 3) Balanced batch sampler (≈50/50)

In [5]:
import math, random
from torch.utils.data import Sampler

class BalancedBatchSampler(Sampler):
    """
    Finite, per-epoch sampler: ~50/50 pos/neg in each batch.
    Steps per epoch = ceil(len(pos) / (batch_size/2)).
    """
    def __init__(self, labels, batch_size=32, pos_label=1, seed=42):
        assert batch_size >= 2 and batch_size % 2 == 0, "batch_size must be even"
        self.labels = list(map(int, labels))
        self.batch_size = batch_size
        self.half = batch_size // 2
        self.pos = [i for i, y in enumerate(self.labels) if y == pos_label]
        self.neg = [i for i, y in enumerate(self.labels) if y == 0]
        if not self.pos or not self.neg:
            raise ValueError("Need both positive and negative samples.")

        self.steps = math.ceil(len(self.pos) / self.half)  # finite!
        self.rng = random.Random(seed)

    def __len__(self):
        return self.steps

    def __iter__(self):
        # shuffle pools each epoch
        pos_pool = self.pos[:]
        neg_pool = self.neg[:]
        self.rng.shuffle(pos_pool)
        self.rng.shuffle(neg_pool)

        pi = 0
        ni = 0
        # ensure neg pool long enough
        # (repeat if needed so we can draw (steps * half) negatives)
        need_neg = self.steps * self.half
        if len(neg_pool) < need_neg:
            reps = math.ceil(need_neg / len(neg_pool))
            neg_pool = (neg_pool * reps)[:need_neg]
        # yield exactly `steps` batches
        for _ in range(self.steps):
            # take half positives
            p_end = pi + self.half
            if p_end <= len(pos_pool):
                p_idx = pos_pool[pi:p_end]
                pi = p_end
            else:
                # wrap and reshuffle remainder to cover all positives once
                rem = len(pos_pool) - pi
                p_idx = pos_pool[pi:]  # remainder
                # reshuffle for the new cycle
                self.rng.shuffle(pos_pool)
                p_idx += pos_pool[: self.half - rem]
                pi = self.half - rem

            # take half negatives
            n_idx = neg_pool[ni:ni+self.half]
            ni += self.half

            batch = p_idx + n_idx
            self.rng.shuffle(batch)
            yield batch


### 4) Model: ResNet18 with 1-ch init

In [44]:
import torchvision.models as models

class FrameClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        try:
            backbone = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        except:
            backbone = models.resnet18(pretrained=True)
        old = backbone.conv1  # (64,3,7,7)
        new = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        with torch.no_grad():
            new.weight.copy_(old.weight.mean(dim=1, keepdim=True))  # average RGB -> gray
        backbone.conv1 = new
        backbone.fc = nn.Linear(backbone.fc.in_features, num_classes)
        self.backbone = backbone

    def forward(self, x):
        return self.backbone(x)


### Model: ResNet50 

In [6]:
import torchvision.models as models

class FrameClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        try:
            backbone = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        except:
            backbone = models.resnet50(pretrained=True)

        # replace first conv (3→1 channels)
        old = backbone.conv1  # (64,3,7,7)
        new = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        with torch.no_grad():
            new.weight.copy_(old.weight.mean(dim=1, keepdim=True))  # average RGB to gray
        backbone.conv1 = new

        # replace classifier head
        backbone.fc = nn.Linear(backbone.fc.in_features, num_classes)

        self.backbone = backbone

    def forward(self, x):
        return self.backbone(x)


### 5) Loss: class-weighted CE (focal optional)

In [6]:
def make_weighted_ce(pos_weight=3.0):
    # weights for classes [0,1]
    w = torch.tensor([1.0, pos_weight], dtype=torch.float32, device=device)
    return nn.CrossEntropyLoss(weight=w)

class FocalLoss(nn.Module):
    def __init__(self, alpha=(1.0, 2.0), gamma=2.0):
        super().__init__()
        self.alpha = torch.tensor(alpha, dtype=torch.float32)
        self.gamma = gamma
    def forward(self, logits, target):
        logp = F.log_softmax(logits, dim=1)
        p = torch.exp(logp)
        at = self.alpha.to(logits.device)[target]
        loss = -at * ((1 - p[range(len(target)), target]) ** self.gamma) * logp[range(len(target)), target]
        return loss.mean()

### 6) Transforms

In [7]:
train_tf = T.Compose([
    T.RandomHorizontalFlip(0.2),
    T.RandomAffine(degrees=10, translate=(0.05, 0.05), scale=(0.9, 1.1)),
    # T.ColorJitter(0.1, 0.1),
    # T.RandomApply([T.GaussianBlur(3)], p=0.2),
    T.Normalize(mean=[0.5], std=[0.5])  # to [-1,1]
])

val_tf = T.Compose([
    T.Normalize(mean=[0.5], std=[0.5])
])

### 7) DataLoaders

In [8]:
from torch.utils.data import DataLoader

TRAIN_DIR = "D:/dataset/npz_80_tiny"
VAL_DIR   = "D:/dataset/converted_classifier_npz_compact"

train_files = sorted([f for f in os.listdir(TRAIN_DIR) if f.endswith(".npz")])[:210]
val_files   = sorted([f for f in os.listdir(VAL_DIR)   if f.endswith(".npz")])[210:255]

# Preload to RAM (FP16). If you’re tight on RAM, reduce out_size or switch dtype=torch.float32 only for val.
train_ds = PreloadedNPZFrameDataset(TRAIN_DIR, train_files, binary=True,
                                    out_size=224, resize_mode="letterbox",
                                    dtype=torch.float16)
val_ds   = PreloadedNPZFrameDataset(VAL_DIR,   val_files,   binary=True,
                                    out_size=224, resize_mode="letterbox",
                                    dtype=torch.float16)

train_loader = DataLoader(
    train_ds,
    batch_sampler=BalancedBatchSampler(train_ds.labels.tolist(), batch_size=16),
    num_workers=0,
    pin_memory=True
)

val_loader = DataLoader(
    val_ds,
    batch_size=16,          # val can use plain batching
    shuffle=False,
    num_workers=0,
    pin_memory=True
)


### 8) Model, Optim, Sched

In [51]:
from torch import amp

model = FrameClassifier(num_classes=2).to(device)

# ---- Phase 1: warmup head only ----
for p in model.backbone.parameters():
    p.requires_grad = False
for p in model.backbone.fc.parameters():
    p.requires_grad = True

criterion = make_weighted_ce(pos_weight=3.0)  # try 2–5
optimizer = torch.optim.Adam(model.backbone.fc.parameters(), lr=LR_HEAD, weight_decay=WEIGHT_DEC)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.5)

scaler = amp.GradScaler(enabled=(device == "cuda"))

best_acc = 0.0
no_improve = 0

def train_one_epoch():
    model.train()
    loop = tqdm(train_loader, desc="Train", total=len(train_loader))
    total_loss = 0.0
    for imgs, lbls in loop:
        # --- move + dtype fix (handles FP16 preloaded tensors) ---
        imgs = imgs.to(device, non_blocking=True).float()   # ensure FP32 inputs
        lbls = lbls.to(device, non_blocking=True)

        # Optional sanity check
        assert imgs.ndim == 4 and imgs.shape[1] == 1, f"Expected [B,1,224,224], got {imgs.shape}"

        optimizer.zero_grad(set_to_none=True)
        with amp.autocast(device_type="cuda", enabled=(device == "cuda")):
            logits = model(imgs)
            loss = criterion(logits, lbls)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        loop.set_postfix(loss=float(loss.item()))
    return total_loss / len(train_loader)

@torch.no_grad()
def validate_frame_level():
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    all_preds, all_labels = [], []
    loop = tqdm(val_loader, desc="Val", total=len(val_loader))
    for imgs, lbls in loop:
        imgs = imgs.to(device, non_blocking=True).float()   # ensure FP32 inputs
        lbls = lbls.to(device, non_blocking=True)

        with amp.autocast(device_type="cuda", enabled=(device == "cuda")):
            logits = model(imgs)
            loss = criterion(logits, lbls)
        val_loss += loss.item()

        preds = logits.argmax(1)
        correct += (preds == lbls).sum().item()
        total += lbls.size(0)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(lbls.cpu().numpy())

    val_loss /= len(val_loader)
    val_acc = 100.0 * correct / total
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    print(classification_report(all_labels, all_preds, target_names=["Background","Positive"], digits=4))
    print("Confusion Matrix:\n", confusion_matrix(all_labels, all_preds))
    return val_acc, val_loss

### 9) Training: Warmup then unfreeze

In [52]:
print("== Phase 1: Warmup head ==")
for epoch in range(3):  # 3–5 warmup epochs
    tl = train_one_epoch()
    val_acc, val_loss = validate_frame_level()
    scheduler.step(val_acc)

# ---- Phase 2: unfreeze all + fine-tune ----
for p in model.backbone.parameters():
    p.requires_grad = True

optimizer = torch.optim.Adam(model.parameters(), lr=LR_ALL, weight_decay=WEIGHT_DEC)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.5)

print("== Phase 2: Fine-tune all ==")
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    tl = train_one_epoch()
    val_acc, val_loss = validate_frame_level()
    scheduler.step(val_acc)

    # early stopping + checkpoint
    if val_acc > best_acc:
        best_acc = val_acc
        no_improve = 0
        torch.save({
            "epoch": epoch,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "best_acc": best_acc
        }, SAVE_PATH)
        print(f"✅ Saved best model (val acc = {best_acc:.2f}%)")
    else:
        no_improve += 1
        if no_improve >= PATIENCE:
            print("⏹️ Early stopping.")
            break

print(f"Training finished. Best validation accuracy = {best_acc:.2f}%")

== Phase 1: Warmup head ==


Val: 100%|██████████| 591/591 [00:08<00:00, 66.35it/s]


Val Loss: 0.7047 | Val Acc: 65.10%
              precision    recall  f1-score   support

  Background     0.9981    0.6432    0.7823     36852
    Positive     0.0642    0.9515    0.1203       948

    accuracy                         0.6510     37800
   macro avg     0.5311    0.7973    0.4513     37800
weighted avg     0.9746    0.6510    0.7657     37800

Confusion Matrix:
 [[23704 13148]
 [   46   902]]


Val: 100%|██████████| 591/591 [00:08<00:00, 68.97it/s]


Val Loss: 0.6790 | Val Acc: 68.36%
              precision    recall  f1-score   support

  Background     0.9974    0.6772    0.8067     36852
    Positive     0.0691    0.9314    0.1287       948

    accuracy                         0.6836     37800
   macro avg     0.5333    0.8043    0.4677     37800
weighted avg     0.9741    0.6836    0.7897     37800

Confusion Matrix:
 [[24958 11894]
 [   65   883]]


...

Val Loss: 0.2002 | Val Acc: 97.46%
              precision    recall  f1-score   support

  Background     0.9941    0.9798    0.9869     36852
    Positive     0.4953    0.7722    0.6035       948

    accuracy                         0.9746     37800
   macro avg     0.7447    0.8760    0.7952     37800
weighted avg     0.9815    0.9746    0.9772     37800

Confusion Matrix:
 [[36106   746]
 [  216   732]]

Epoch 19/20


Val: 100%|██████████| 591/591 [00:08<00:00, 68.49it/s]


Val Loss: 0.2027 | Val Acc: 97.40%
              precision    recall  f1-score   support

  Background     0.9945    0.9788    0.9866     36852
    Positive     0.4892    0.7911    0.6046       948

    accuracy                         0.9740     37800
   macro avg     0.7419    0.8849    0.7956     37800
weighted avg     0.9819    0.9740    0.9770     37800

Confusion Matrix:
 [[36069   783]
 [  198   750]]

Epoch 20/20


Val: 100%|██████████| 591/591 [00:08<00:00, 68.50it/s]

Val Loss: 0.1955 | Val Acc: 96.99%
              precision    recall  f1-score   support

  Background     0.9951    0.9739    0.9844     36852
    Positive     0.4448    0.8122    0.5748       948

    accuracy                         0.9699     37800
   macro avg     0.7199    0.8931    0.7796     37800
weighted avg     0.9813    0.9699    0.9741     37800

Confusion Matrix:
 [[35891   961]
 [  178   770]]
⏹️ Early stopping.
Training finished. Best validation accuracy = 97.51%


In [9]:
from torch import amp

model = FrameClassifier(num_classes=2).to(device)

# ---- Phase 1: warmup head only ----
for p in model.backbone.parameters():
    p.requires_grad = False
for p in model.backbone.fc.parameters():
    p.requires_grad = True

criterion = make_weighted_ce(pos_weight=3.0)  # try 2–5
optimizer = torch.optim.Adam(model.backbone.fc.parameters(), lr=LR_HEAD, weight_decay=WEIGHT_DEC)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.5)

scaler = amp.GradScaler(enabled=(device == "cuda"))

best_acc = 0.0
no_improve = 0

def train_one_epoch():
    model.train()
    loop = tqdm(train_loader, desc="Train", total=len(train_loader))
    total_loss = 0.0
    for imgs, lbls in loop:
        # --- move + dtype fix (handles FP16 preloaded tensors) ---
        imgs = imgs.to(device, non_blocking=True).float()   # ensure FP32 inputs
        lbls = lbls.to(device, non_blocking=True)

        # Optional sanity check
        assert imgs.ndim == 4 and imgs.shape[1] == 1, f"Expected [B,1,224,224], got {imgs.shape}"

        optimizer.zero_grad(set_to_none=True)
        with amp.autocast(device_type="cuda", enabled=(device == "cuda")):
            logits = model(imgs)
            loss = criterion(logits, lbls)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        loop.set_postfix(loss=float(loss.item()))
    return total_loss / len(train_loader)

@torch.no_grad()
def validate_frame_level():
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    all_preds, all_labels = [], []
    loop = tqdm(val_loader, desc="Val", total=len(val_loader))
    for imgs, lbls in loop:
        imgs = imgs.to(device, non_blocking=True).float()   # ensure FP32 inputs
        lbls = lbls.to(device, non_blocking=True)

        with amp.autocast(device_type="cuda", enabled=(device == "cuda")):
            logits = model(imgs)
            loss = criterion(logits, lbls)
        val_loss += loss.item()

        preds = logits.argmax(1)
        correct += (preds == lbls).sum().item()
        total += lbls.size(0)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(lbls.cpu().numpy())

    val_loss /= len(val_loader)
    val_acc = 100.0 * correct / total
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    print(classification_report(all_labels, all_preds, target_names=["Background","Positive"], digits=4))
    print("Confusion Matrix:\n", confusion_matrix(all_labels, all_preds))
    return val_acc, val_loss

In [10]:
print("== Phase 1: Warmup head ==")
for epoch in range(3):  # 3–5 warmup epochs
    tl = train_one_epoch()
    val_acc, val_loss = validate_frame_level()
    scheduler.step(val_acc)

# ---- Phase 2: unfreeze all + fine-tune ----
for p in model.backbone.parameters():
    p.requires_grad = True

optimizer = torch.optim.Adam(model.parameters(), lr=LR_ALL, weight_decay=WEIGHT_DEC)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.5)

print("== Phase 2: Fine-tune all ==")
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    tl = train_one_epoch()
    val_acc, val_loss = validate_frame_level()
    scheduler.step(val_acc)

    # early stopping + checkpoint
    if val_acc > best_acc:
        best_acc = val_acc
        no_improve = 0
        torch.save({
            "epoch": epoch,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "best_acc": best_acc
        }, SAVE_PATH)
        print(f"✅ Saved best model (val acc = {best_acc:.2f}%)")
    else:
        no_improve += 1
        if no_improve >= PATIENCE:
            print("⏹️ Early stopping.")
            break

print(f"Training finished. Best validation accuracy = {best_acc:.2f}%")

== Phase 1: Warmup head ==


Val: 100%|██████████| 2363/2363 [00:24<00:00, 96.97it/s] 


Val Loss: 0.6434 | Val Acc: 69.79%
              precision    recall  f1-score   support

  Background     0.9982    0.6914    0.8169     36852
    Positive     0.0734    0.9504    0.1363       948

    accuracy                         0.6979     37800
   macro avg     0.5358    0.8209    0.4766     37800
weighted avg     0.9750    0.6979    0.7999     37800

Confusion Matrix:
 [[25480 11372]
 [   47   901]]


Val: 100%|██████████| 2363/2363 [00:22<00:00, 103.12it/s]


Val Loss: 0.8229 | Val Acc: 65.32%
              precision    recall  f1-score   support

  Background     0.9983    0.6454    0.7839     36852
    Positive     0.0650    0.9578    0.1217       948

    accuracy                         0.6532     37800
   macro avg     0.5316    0.8016    0.4528     37800
weighted avg     0.9749    0.6532    0.7673     37800

Confusion Matrix:
 [[23783 13069]
 [   40   908]]


...

Val Loss: 0.1179 | Val Acc: 96.82%
              precision    recall  f1-score   support

  Background     0.9942    0.9730    0.9835     36852
    Positive     0.4265    0.7806    0.5516       948

    accuracy                         0.9682     37800
   macro avg     0.7104    0.8768    0.7676     37800
weighted avg     0.9800    0.9682    0.9727     37800

Confusion Matrix:
 [[35857   995]
 [  208   740]]

Epoch 9/20


Val: 100%|██████████| 2363/2363 [00:22<00:00, 102.80it/s]


Val Loss: 0.1232 | Val Acc: 96.88%
              precision    recall  f1-score   support

  Background     0.9939    0.9740    0.9839     36852
    Positive     0.4317    0.7669    0.5524       948

    accuracy                         0.9688     37800
   macro avg     0.7128    0.8705    0.7681     37800
weighted avg     0.9798    0.9688    0.9730     37800

Confusion Matrix:
 [[35895   957]
 [  221   727]]

Epoch 10/20


Val: 100%|██████████| 2363/2363 [00:23<00:00, 102.24it/s]

Val Loss: 0.1255 | Val Acc: 97.12%
              precision    recall  f1-score   support

  Background     0.9943    0.9760    0.9851     36852
    Positive     0.4569    0.7838    0.5773       948

    accuracy                         0.9712     37800
   macro avg     0.7256    0.8799    0.7812     37800
weighted avg     0.9809    0.9712    0.9749     37800

Confusion Matrix:
 [[35969   883]
 [  205   743]]
⏹️ Early stopping.
Training finished. Best validation accuracy = 97.17%


In [10]:
import os, numpy as np, torch
import torch.nn.functional as F
from torch import amp
from tqdm import tqdm

# -----------------------------
# Config (edit these)
# -----------------------------
DEVICE      = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH  = "D:/acouslic-ai-cse4622/saved_weights/best_frame_classifier_allinone.pth"
TEST_DIR    = "D:/dataset/converted_classifier_npz_compact"   # full 840-frame npz files
START_IDX   = 255   # where your test split starts
NUM_CASES   = 45
IMAGE_SIZE  = 224
MA_WINDOW   = 7     # temporal smoothing (moving average)
TOPK        = 1     # use top-1 for "best frame"
THRESH      = None  # or e.g. 0.6 to require minimum smoothed prob
TOLERANCE   = 0     # 0 = exact frame; try 3 for ±3 frames

# -----------------------------
# Utils (match your training preprocessing)
# -----------------------------
def letterbox_to_square_tensor(x: torch.Tensor, size=IMAGE_SIZE) -> torch.Tensor:
    """x: (B,1,H,W) or (1,H,W) in [0,1] -> square pad -> resize to (..,1,size,size)"""
    is_batched = (x.dim() == 4)
    if not is_batched: x = x.unsqueeze(0)
    _, _, H, W = x.shape
    s = max(H, W)
    pad_h = (s - H) // 2
    pad_w = (s - W) // 2
    x = F.pad(x, (pad_w, s-W-pad_w, pad_h, s-H-pad_h))
    x = F.interpolate(x, size=(size, size), mode="bilinear", align_corners=False)
    return x if is_batched else x.squeeze(0)

def moving_average(a: np.ndarray, w: int = 7):
    if w <= 1: return a
    k = np.ones(w, dtype=np.float32) / w
    return np.convolve(a, k, mode="same")

@torch.no_grad()
def predict_probs_over_frames(npz_path, model, batch_size=128):
    d = np.load(npz_path, mmap_mode="r")
    frames = d["image"]  # (T,H,W) uint8
    Tn = len(frames)
    probs = np.zeros(Tn, dtype=np.float32)
    off = 0
    while off < Tn:
        chunk = frames[off:off+batch_size]
        x = torch.from_numpy(chunk).unsqueeze(1).float() / 255.0      # (B,1,H,W) [0,1]
        x = letterbox_to_square_tensor(x, size=IMAGE_SIZE)            # (B,1,S,S)
        x = (x - 0.5) / 0.5                                           # [-1,1]
        x = x.to(DEVICE)

        with amp.autocast(device_type="cuda", enabled=(DEVICE=="cuda")):
            logits = model(x)
            p = torch.softmax(logits, dim=1)[:, 1].detach().cpu().numpy()
        probs[off:off+len(p)] = p
        off += len(p)
    return probs

@torch.no_grad()
def pick_best_frame(npz_path, model, ma_window=7, topk=1, thresh=None):
    """Returns best_idx (int), best_score (float), idx_topk (np.ndarray), raw_probs, smoothed_probs."""
    probs = predict_probs_over_frames(npz_path, model)
    sm = moving_average(probs, w=ma_window)
    idx_sorted = np.argsort(-sm)
    idx_topk = idx_sorted[:topk]
    if thresh is not None:
        idx_topk = np.array([i for i in idx_topk if sm[i] >= thresh], dtype=int)
        if len(idx_topk) == 0:
            idx_topk = np.array([int(np.argmax(sm))], dtype=int)
    best_idx = int(idx_topk[0])
    best_score = float(sm[best_idx])
    return best_idx, best_score, idx_topk, probs, sm

def load_binary_labels(npz_path):
    y = np.load(npz_path, mmap_mode="r")["label"].astype(np.int64)
    y[y == 2] = 1
    return y

# -----------------------------
# Load model
# -----------------------------
model = FrameClassifier(num_classes=2).to(DEVICE)
ckpt = torch.load(MODEL_PATH, map_location=DEVICE)
state = ckpt.get("model_state", ckpt)  # support plain state_dict too
model.load_state_dict(state)
model.eval()

# -----------------------------
# Pick 45 test files and evaluate
# -----------------------------
files = sorted([f for f in os.listdir(TEST_DIR) if f.endswith(".npz")])
test_files = files[START_IDX:START_IDX+NUM_CASES]
assert len(test_files) > 0, "No test files selected."

correct = 0
details = []

print(f"Evaluating {len(test_files)} cases (tolerance=±{TOLERANCE})...\n")
for f in tqdm(test_files):
    path = os.path.join(TEST_DIR, f)
    y = load_binary_labels(path)
    pos_idx = np.where(y == 1)[0]  # ground truth positive frames

    best_idx, best_score, idx_topk, probs, sm = pick_best_frame(
        path, model, ma_window=MA_WINDOW, topk=TOPK, thresh=THRESH
    )

    if len(pos_idx) == 0:
        hit = (y[best_idx] == 0)  # no positives: "correct" if best is background
        dist = 0
    else:
        # exact or ±tolerance match
        dist = int(np.min(np.abs(pos_idx - best_idx)))
        hit = (dist <= TOLERANCE)

    correct += int(hit)
    details.append((f, best_idx, best_score, dist, hit))

# -----------------------------
# Summary
# -----------------------------
print("\n====== Summary ======")
print(f"Correct best-frame picks: {correct}/{len(test_files)} ({100.0*correct/len(test_files):.1f}%)")
print(f"(tolerance = ±{TOLERANCE}, MA={MA_WINDOW}, topk={TOPK}, thresh={THRESH})")

# Optional: show a few per-case lines
for f, bi, bs, d, h in details[:45]:
    print(f"{f}: best={bi:4d}  prob={bs:.3f}  dist_to_pos={d:3d}  {'HIT' if h else 'MISS'}")


Evaluating 45 cases (tolerance=±0)...



100%|██████████| 45/45 [00:42<00:00,  1.05it/s]


====== Summary ======
Correct best-frame picks: 38/45 (84.4%)
(tolerance = ±0, MA=7, topk=1, thresh=None)
d42fb920-5df1-4341-93df-480c17355e44.npz: best= 804  prob=0.986  dist_to_pos=  0  HIT
d5471cfd-6090-4d42-9a95-67ccbfbf612e.npz: best=  48  prob=0.922  dist_to_pos=  0  HIT
d571d4e1-ff80-44b9-a481-07961c6a1208.npz: best=  52  prob=0.987  dist_to_pos=  0  HIT
d5c3cfee-53ac-4021-8c1b-098c189f630e.npz: best= 623  prob=0.997  dist_to_pos=  0  HIT
d5f8c859-de93-4a50-b324-1ae4ad0267d4.npz: best= 208  prob=0.928  dist_to_pos=  0  HIT
d624338f-d09b-4bda-bbc3-3fa417015d6b.npz: best=  78  prob=0.999  dist_to_pos=  0  HIT
d77b6ece-da17-4f88-818c-0c7340b3e54f.npz: best=  58  prob=0.999  dist_to_pos=  0  HIT
d812091a-3635-4d51-9290-6adb3aa8681e.npz: best=  37  prob=0.979  dist_to_pos=  0  HIT
d8c3665a-4dc3-40ce-b716-f30aab365332.npz: best= 181  prob=0.578  dist_to_pos=410  MISS
db9d468d-cb20-4d5e-b059-31728f5950e6.npz: best=  50  prob=0.973  dist_to_pos=  0  HIT
dc0cbbdf-e4bb-4de5-958a-10576129